# Socrata API Project - Nashville.gov Top 500 Monthly Searches

In [ ]:
import requests
import matplotlib.pyplot as plt
import pandas as pd

### Q1. Make an API request that returns the months where "fire" was searched in 2016. Which month had the most searches?

In [ ]:
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?year=2016&query_text=fire'
response = requests.get(endpoint)

In [ ]:
# Rudy: Can use parameters to pull the same thing
# endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json'
# params = {
#         "year": "2016",
#         "query_text": "fire"    
#     }
# fires2016 = requests.get(endpoint, params = params)
# fires2016.text

In [ ]:
response

In [ ]:
res_2016_fire = response.json()
res_2016_fire

In [ ]:
# Order results by query_count DESC to get the max query_count first
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?year=2016&query_text=fire&$order=query_count DESC'
response = requests.get(endpoint)
max_fire = response.json()
max_fire

In [ ]:
# Limit 1 to get the month with the top query_count for 'fire'
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?year=2016&query_text=fire&$order=query_count DESC&$limit=1'
response = requests.get(endpoint)
max_fire = response.json()
max_fire

#### A1. August had the most searches for 'fire' at 47.

### Q2. Make an API request that returns all the times a query was run more than 100 times in a month. How many times did this occur?

In [ ]:
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?$where=query_count > 100'
response = requests.get(endpoint)
over100 = response.json()
# over100

In [ ]:
times = len(over100)
times
# 1,000 looks too exact... investigating found this:
# https://stackoverflow.com/questions/30941170/why-am-i-limited-to-1-000-rows-on-soda-api-when-i-have-an-app-key
# It defaults to 1,000 records if limit is not specified

In [ ]:
# I can specify a limit OR use count:
# https://stackoverflow.com/questions/27823200/is-there-a-socrata-api-method-to-get-the-row-count-of-a-dataset
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?$select=count(*)&$where=query_count > 100'
response = requests.get(endpoint)
over100_count = response.json()
over100_count

#### A2. 1,262 is the number of times a query was run more than 100 times in a month.

### Q3. Make another API request that returns all the times "codes" was searched more than 100 times in a month. How many times did this occur?

In [ ]:
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?$select=query_text&$where=query_count > 100 AND query_text=codes'
response = requests.get(endpoint)
over100_codes_count = response.json()
over100_codes_count
# Not working...

In [ ]:
# Omitting select:
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?$where=query_count > 100 AND query_text=codes'
response = requests.get(endpoint)
over100_codes_count = response.json()
over100_codes_count
# Still thinks codes is a column, maybe the where statement is wrong?

In [ ]:
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?query_text=codes&query_count > 100'
response = requests.get(endpoint)
over100_codes_count = response.json()
over100_codes_count
# A new error message means I'm getting closer right?!

In [ ]:
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?query_text=codes&$where=query_count > 100'
response = requests.get(endpoint)
over100_codes_count = response.json()
len(over100_codes_count)

In [ ]:
# How others did it:

# Patrick: $select=count(*)&$where=query_count > 100&query_text=codes'
# Maggie: $query=select count(month_name) where query_count>100 AND query_text="codes"'

#### A3. 56 is the number of times "codes" was searched more than 100 times in a month.

### Q4. Make an API request that returns the entire Top 500 Monthly Searches dataset. Make a chart that shows the number of times "maps" was searched in a month across the entire time frame.

#### Interpretation 1: Find the number of times "maps" was searched month by month across the entire time frame.

In [ ]:
# Let's check how many records are in this dataset
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?$select=count(*)'
response = requests.get(endpoint)
count = response.json()
count

In [ ]:
# 43,677 records fit within the max limit parameter of 50k
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?$limit=50000'
response = requests.get(endpoint)
all = response.json()
#all

In [ ]:
# Same as above but with query_text=maps
# https://stackoverflow.com/questions/59529516/import-all-rows-from-dataset-using-soda-api-python
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json?query_text=maps&$limit=50000'
response = requests.get(endpoint)
maps = response.json()
df = pd.DataFrame(maps)
print(df.shape)

In [ ]:
df.head()

In [ ]:
# Remove query_text and month columns
df = df.drop(columns=['query_text', 'month'], axis=1)
df.head()

In [ ]:
# Adding a new col to combine month and year for x-axis
df["monthyear"] = df["month_name"] + df["year"]
df.head()

In [ ]:
# Convert query_count to numeric
df["query_count"] = pd.to_numeric(df["query_count"])

In [ ]:
df.plot(x="monthyear", y="query_count", figsize=(18, 9))
plt.title("Number of Times 'Maps' was Searched Across Entire Dataset", fontsize=20)
plt.xlabel("Month", fontsize=15)
plt.ylabel("Number of times 'maps' was searched", fontsize=15)
plt.show()

#### Interpretation 2: Find the number of times "maps" was searched for EACH MONTH across the entire time frame.

In [ ]:
# Abigail: '?$select=month_name,sum(query_count)&$group=month_name&$where=query_text="maps"&$limit=50000'

In [ ]:
endpoint_maps = 'https://data.nashville.gov/resource/fuaa-r5cm.json?query_text=maps&$limit=50000'
response_maps = requests.get(endpoint_maps)
maps2_df = pd.read_json(endpoint_maps)
print(maps2_df.to_string()) 

In [ ]:
maps2_df = pd.DataFrame(maps2_df)
maps2_df

In [ ]:
# https://stackoverflow.com/questions/59242019/pandas-sum-all-rows-with-the-same-month
maps2_df = maps2_df.groupby(['month_name'])[['query_count']].sum().reset_index()
maps2_df

In [ ]:
maps2_df["query_count"] = pd.to_numeric(maps2_df["query_count"])
maps2_df

In [ ]:
maps2_df.sort_values(by=['query_count'], inplace=True, ascending=False)
maps2_df

In [ ]:
plt.figure(figsize=(18, 9)) # Figsize goes first
plt.bar(maps2_df.month_name, maps2_df.query_count)
plt.xlabel("Month", fontsize=15)
plt.ylabel("Search Count", fontsize=15)
plt.title("Total Searches for Map by Month", fontsize=20)
plt.xticks(rotation = 45)
plt.show();
# Thanks to Rudy for sharing his plot code

Here is the result for the second interpretation. Big thanks to my group for helping me plot this (in order)!

### Q5. Make an API request to pull back all the data from hubNashville (311) Service Requests (check to see how many rows you can return in a single request). Compare it to the Top 500 Monthly Searches data set. What do you observe? (This is open-ended, there isn't a specific answer for this one)

In [ ]:
endpoint = 'https://data.nashville.gov/resource/7qhx-rexh.json?$select=count(*)&$limit=1000000' # don't need last part
response = requests.get(endpoint)
service = response.json()
service

#### A5. This dataset has 936,864 rows compared to the 43,677 rows from the Top 500 Monthly Searches data set.

### Q6. Find 2 new data sets on data.nashville.gov, make API requests to pull the data, and do an analysis that combines the data sets.

Building Permit Applications & Building Permits Issued for Nashville

In [ ]:
# Building Permit Applications
endpoint1 = 'https://data.nashville.gov/resource/kqff-rxj8.json?city=NASHVILLE'
response = requests.get(endpoint1)
nash_bdg_apps = response.json()
apps_df = pd.DataFrame(nash_bdg_apps)
apps_df.head()

In [ ]:
# Building Permits Issued
endpoint2 = 'https://data.nashville.gov/resource/3h5w-q8b7.json?city=NASHVILLE'
response = requests.get(endpoint2)
nash_bdg_permits = response.json()
permits_df = pd.DataFrame(nash_bdg_permits)
permits_df.head()

Our group ended up talking and never got to finishing this...